In [0]:
from pyspark.sql import Window

In [0]:
from pyspark.sql.functions import *

In [0]:
class Transformer:
    def transform(self, inputDFs):    
        pass


class CustomerTransactionTransformater(Transformer):
    def transform(self, inputDFs):
        transactionDF = inputDFs["transactionInputDF"]
        customerDF = inputDFs["customerInputDF"]
   
        windowSpec = Window.partitionBy("customer_id").orderBy("transaction_date")

        # Transformation: Adding lead and lag
        transactionWithLeadLagDF = transactionDF.withColumn(
            "Next_product", lead("product_name", 1).over(windowSpec)
        ).withColumn(
            "Previous_product", lag("product_name", 1).over(windowSpec)
        )

        print("Transactions Data with Lead and Lag:")
        transactionWithLeadLagDF.show()

        # Business Logic 1: Customers who bought AirPods after buying iPhone
        airpodsAfterIphoneDF = transactionWithLeadLagDF.filter(
            (col("product_name") == "iPhone") & (col("Next_product") == "AirPods")
        ).select("customer_id").distinct()

        # Result
        airPodResultDF = airpodsAfterIphoneDF.join(customerDF, "customer_id")

        # Business Logic 2: Customers who bought both iPhone and AirPods
        airpodsIphoneDF = transactionDF.groupBy("customer_id").agg(
            countDistinct("product_name").alias("product_count")
        ).filter(col("product_count") == 2)

        # Result 2
        airpodsIphoneResultDF = airpodsIphoneDF.join(customerDF, "customer_id")

        return airPodResultDF, airpodsIphoneResultDF

   